<a href="https://colab.research.google.com/github/kinjaljoshi/ma_rag_components/blob/master/decisioning_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Code to show routing based upon Object routing registry

In [ ]:
!pip install langchain_community

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
from google.colab import userdata

# === Object Routing Registry ===
object_routing_registry = {
    "purchase_order": {
        "access_method": "api",
        "endpoint": "/api/purchase-orders"
    },
    "purchase_order_status": {
        "access_method": "api",
        "endpoint": "/api/purchase-orders/status"
    },
    "__default__": {
        "access_method": "sql"
    }
}
openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key

# === Prompt Template ===
routing_prompt = PromptTemplate.from_template("""
You are a smart assistant helping route user queries to backend services.
Given the following list of objects with access methods:

{object_descriptions}

Determine the best matching object for this user query:
"{user_query}"

Respond only with the exact object name from the list above, or "__default__" if nothing matches.
""")

# === LLM Setup ===
llm = ChatOpenAI(temperature=0)
routing_chain = LLMChain(llm=llm, prompt=routing_prompt)

# === Helper Function: Convert Registry to Input String ===
def format_object_descriptions(routing_registry: dict) -> str:
    descriptions = []
    for name, conf in routing_registry.items():
        if name == "__default__":
            descriptions.append(f"- {name} (fallback if no match)")
        else:
            endpoint = conf.get("endpoint", "N/A")
            method = conf.get("access_method", "unknown")
            descriptions.append(f"- {name} (access_method: {method}, endpoint: {endpoint})")
    return "\n".join(descriptions)

# === Manager Agent Routing Logic Using LLM ===
def manager_routing_logic_llm(query: str) -> str:
    object_descriptions = format_object_descriptions(object_routing_registry)
    response = routing_chain.run({
        "object_descriptions": object_descriptions,
        "user_query": query
    }).strip()
    print(f" LLM selected method: {response}")
    return object_routing_registry.get(response, object_routing_registry["__default__"])["access_method"]


In [14]:
query_1 = "What's the status of my purchase orders?"
print("Access method:", manager_routing_logic_llm(query_1))

query_2 = "Summarize sales by region"
print("Access method:", manager_routing_logic_llm(query_2))


 LLM selected method: purchase_order_status
Access method: api
 LLM selected method: __default__
Access method: sql
